## Convolutional NN to classify govuk content to level2 taxons

Based on:
https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

## To do:
- ~~Consider grouping very small classes (especially if too small for evaluation)~~
- ~~Split data into training, validation and test to avoid overfitting validation data during hyperparamter searches & model architecture changes~~
- ~~Try learning embeddings~~--
- ~~Try changing pos_ratio~~
- Try implementing class_weights during model fit (does this do the same as the weighted binary corss entropy?)
- Work on tensorboard callbacks
- ~~Create dictionary of class indices to taxon names for viewing results~~
- ~~Check model architecture~~
- ~~consider relationship of training error to validation error - overfitting/bias?~~
- ~~train longer~~
- Try differnet max_sequence_length
- Check batch size is appropriate
- Also think about:
  - ~~regularization (e.g. dropout)~~ 
  - fine-tuning the Embedding layer

### Load requirements and data

TODO: edit requirement.txt to include only these packages and do not include tensorflow because this conflicts with tf on AWS when using on GPU.

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical, layer_utils, plot_model

from keras.layers import (Embedding, Input, Dense, Dropout, 
                          Activation, Conv1D, MaxPooling1D, Flatten, concatenate, Reshape)
from keras.models import Model, Sequential
from keras.optimizers import rmsprop
from keras.callbacks import TensorBoard, Callback, ModelCheckpoint
import keras.backend as K
from keras.losses import binary_crossentropy

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score 
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.utils import class_weight

import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

import functools

import h5py

import time

/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


### Environmental vars

In [2]:
DATADIR=os.getenv('DATADIR')
#DATADIR='/data' #this was put in for AWS run but doesn't work locally...

## Hyperparameters

Intuition for POS_RATIO is that it penalises the prediction of zero for everything, which is attractive to the model because the multilabel y matrix is super sparse. 

Increasing POS_RATIO should penalise predicting zeros more.

In [3]:
#MAX_NB_WORDS
MAX_SEQUENCE_LENGTH =1000
EMBEDDING_DIM = 100 # keras embedding layer output_dim = Dimension of the dense embedding
P_THRESHOLD = 0.5 #Threshold for probability of being assigned to class
POS_RATIO = 0.5 #ratio of positive to negative for each class in weighted binary cross entropy loss function
NUM_WORDS=20000 #keras tokenizer num_words: None or int. Maximum number of words to work with 
#(if set, tokenization will be restricted to the top num_words most common words in the dataset).

### Read in data
Content items tagged to level 2 taxons or lower in the topic taxonomy

In [4]:
labelled_level2 = pd.read_csv(os.path.join(DATADIR, 'labelled_level2.csv.gz'), dtype=object, compression='gzip')

In [5]:
labelled_level2.shape

(172916, 21)

In [6]:
labelled_level2['content_id'].nunique()

113481

#### clean up any World taxons leftover despite dropping relevant doctypes

In [7]:
#COLLAPSE World level2taxons
labelled_level2.loc[labelled_level2['level1taxon'] == 'World', 'level2taxon'] = 'world_level1'

#creating categorical variable for level2taxons from values
labelled_level2['level2taxon'] = labelled_level2['level2taxon'].astype('category')

In [8]:
#count the number of content items per taxon into new column
labelled_level2['num_content_per_taxon'] = labelled_level2.groupby(["level2taxon"])['level2taxon'].transform("count")

In [9]:
labelled_level2['num_content_per_taxon'].describe()

count    172916.000000
mean       4575.345752
std        3691.673143
min           1.000000
25%        1500.000000
50%        3780.000000
75%        6156.000000
max       11717.000000
Name: num_content_per_taxon, dtype: float64

In [10]:
#number of rows in biggest level2 taxon -this is the target size for all other level2 taxons in resampling
max_content_freq = max(labelled_level2['num_content_per_taxon'])
max_content_freq

11717

### drop news

In [11]:
labelled_level2.shape

(172916, 22)

In [12]:
labelled_level2[(labelled_level2['document_type'] == 'world_news_story')].shape

(3927, 22)

In [13]:
labelled_level2[(labelled_level2['document_type'] == 'news_story')].shape

(33214, 22)

In [14]:
nonews = labelled_level2[(labelled_level2['document_type'] != 'news_story') &
                         (labelled_level2['document_type'] != 'world_news_story')]

In [15]:
nonews.shape

(135775, 22)

### Create dictionary mapping taxon codes to string labels

In [16]:
#Get the category numeric values (codes) and avoid zero-indexing
labels = nonews['level2taxon'].cat.codes + 1

#create dictionary of taxon category code to string label for use in model evaluation
labels_index = dict(zip((labels), nonews['level2taxon']))

In [17]:
#labels_index

In [18]:
print(len(labels_index))

210


### Create target/Y 

Note: when using the categorical_crossentropy loss, your targets should be in categorical format (e.g. if you have 10 classes, the target for each sample should be a 10-dimensional vector that is all-zeros expect for a 1 at the index corresponding to the class of the sample).

In multilabel learning, the joint set of binary classification tasks is expressed with label binary indicator array: each sample is one row of a 2d array of shape (n_samples, n_classes) with binary values:  
the one, i.e. the non zero elements, corresponds to the subset of labels.  
An array such as np.array([[1, 0, 0], [0, 1, 1], [0, 0, 0]]) represents label 0 in the first sample, labels 1 and 2 in the second sample, and no labels in the third sample.  
Producing multilabel data as a list of sets of labels may be more intuitive.

####  First reshape wide to get columns for each level2taxon and row number = number unique urls

In [19]:
#get a smaller copy of data for pivoting ease (think you can work from full data actually and other cols get droopedauto)

level2_reduced = nonews[['content_id', 
                         'level2taxon', 
                         'combined_text', 
                         'title', 
                         'description',
                         'document_type', 
                            'first_published_at', 
                            'publishing_app', 
                            'primary_publishing_organisation']].copy()

#how many level2taxons are there?
print('Number of unique level2taxons: {}'.format(level2_reduced.level2taxon.nunique()))

#count the number of taxons per content item into new column
level2_reduced['num_taxon_per_content'] = level2_reduced.groupby(["content_id"])['content_id'].transform("count")

#Add 1 because of zero-indexing to get 1-number of level2taxons as numerical targets
level2_reduced['level2taxon_code'] = level2_reduced.level2taxon.astype('category').cat.codes + 1

Number of unique level2taxons: 210


In [20]:
#how many level2taxons are there?
print('Number of unique level2taxons: {}'.format(labelled_level2.level2taxon.nunique()))

#count the number of taxons per content item into new column
labelled_level2['num_taxon_per_content'] = labelled_level2.groupby(["content_id"])['content_id'].transform("count")

#Add 1 because of zero-indexing to get 1-number of level2taxons as numerical targets
labelled_level2['level2taxon_code'] = labelled_level2.level2taxon.astype('category').cat.codes + 1

Number of unique level2taxons: 210


In [21]:
#reshape to wide per taxon and keep the combined text so indexing is consistent when splitting X from Y

multilabel = (level2_reduced.pivot_table(index=['content_id', 
                                                'combined_text', 
                                                'title', 
                                                'description' 
                                                ] , columns='level2taxon_code', values='num_taxon_per_content'))
print('level2reduced shape: {}'.format(level2_reduced.shape))
print('pivot table shape (no duplicates): {} '.format(multilabel.shape))


level2reduced shape: (135775, 11)
pivot table shape (no duplicates): (91771, 210) 


In [22]:
multilabel.columns

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            201, 202, 203, 204, 205, 206, 207, 208, 209, 210],
           dtype='int64', name='level2taxon_code', length=210)

In [23]:
multilabel.head()

,,,level2taxon_code,1,2,3,4,5,6,7,8,9,10,...,201,202,203,204,205,206,207,208,209,210
content_id,combined_text,title,description,,,,,,,,,,,,,,,,,,,,,
00029fa4-9b60-4285-898c-85ae8a6367f5,emma jones - small business crown representative as small business crown representative emma is keen to help uk smes win government business. emma was appointed as small business crown representative in july 2016. she was selected for the role because of her wealth of experience in working with smes. she is the founder of small business support group enterprise nation and the co founder of startup britain. emma’s work in her role as small business crown representative includes: working with government and the small business panel to identify the remaining barriers to smes doing business with the public sector supporting the launch and delivery of the campaign to help show that government is “open for business” for smes and helping them bid for and win more contracts increasing awareness among smaller businesses of opportunities to deliver on behalf of larger private sector firms who have secured government contracts working with government to identify new opportunities to get best value from smes getting support emma is keen to hear what small business have to say and wants to engage with as many smes as possible. so if you’re thinking about becoming a government supplier take a look at the events and opportunities below for how to get involved and gain support. events the leeds cross government sme roadshow 24 november 2017 is a great opportunity for smes to hear directly about the opportunities to sell to the public sector. more information about the event and how to register can be found here. webinars register for free for emma’s half hour webinars offering advice on how to become a government supplier. a list of webinars coming up is featured below. blogs read emma’s blogs to gain useful insight updates and tips for smes and government buyers. these smes did it and so can you! prompt payment makes for good business 2017 a big year for small businesses calling central government buyers: emma can help you meet your target small business saturday dec 2016: top tips for selling to government selling to the public sector guide in partner with the crown commercial service emma has developed a guide for small businesses with tips on selling to government. read here . government is open for business ‘open for business’ is the government’s campaign to reach more smes as potential suppliers: to help and support them to become suppliers and to listen to how government can improve the process. for more information visit www.gov.uk/openforbusiness register with contracts finder to keep updated on new and upcoming contracts worth over £10 000. for inspiration on how other small business have grown and benefitted from being a supplier government read our case studies . if you would like to help in getting the message out that government is open for business then visit the resources page for ways in which you can support.,emma jones - small business crown representative,as small business crown representative emma is keen to help uk smes win government business.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00037b70-5b08-44c2-bf0a-fa8eb636a60b,land remediation: bringing brownfield sites back to use brochure showing uk expertise in land remediation outlining technologies systems and ideas used in the regeneration of industrial land. the uk was the first industrialised country in the world. the legacy of the industrial revolution is over 400 000 hectares of contaminated land. uk expertise in land remediation has been borne out of necessity. the department for international trade’s ( dit ) brochure provides an overview of the expertise gained from over 5 decades of experience in land remediation. the brochure includes information on: sector specialists urban regeneration spill response monitoring and validation corporate 

In [24]:
multilabel.columns.astype('str')

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       ...
       '201', '202', '203', '204', '205', '206', '207', '208', '209', '210'],
      dtype='object', name='level2taxon_code', length=210)

In [25]:
#THIS IS WHY INDEXING IS NOT ZERO-BASED
#convert the number_of_taxons_per_content values to 1, meaning there was an entry for this taxon and this content_id, 0 otherwise
binary_multilabel = multilabel.notnull().astype('int')

## Data Pre-Processing

In [26]:
total_size = binary_multilabel.shape[0]
total_size

91771

In [27]:
nb_test_samples = int(0.1 * total_size) #test split
print('nb_test samples:', nb_test_samples)

nb_dev_samples = int(0.2 * total_size) #dev split
print('nb_dev samples:', nb_dev_samples)

nb_training_samples = int(0.8 * total_size) #train split
print('nb_training samples:', nb_training_samples)

nb_test samples: 9177
nb_dev samples: 18354
nb_training samples: 73416


### Shuffle

In [28]:
for i in range(0,10):
    print(binary_multilabel.index[i][0])

00029fa4-9b60-4285-898c-85ae8a6367f5
00037b70-5b08-44c2-bf0a-fa8eb636a60b
00037ee5-7b5e-452d-a233-af2c134f5bce
0004c63d-ae16-432a-bb35-c0f949b1e27c
0005ac76-50fe-42f1-8168-8b6fc046e40f
0006811c-ad80-4cd0-a732-04cc983ec8c2
0008f82f-9713-4074-8793-0d266d53930c
000aa34d-c3c0-4176-ad8a-50e801056df1
000b6a38-c69a-4ac9-918b-717a79cbdad2
000b8c7e-4671-4586-9eff-97c0c374126b


00029fa4-9b60-4285-898c-85ae8a6367f5
00037b70-5b08-44c2-bf0a-fa8eb636a60b
00037ee5-7b5e-452d-a233-af2c134f5bce
0004c63d-ae16-432a-bb35-c0f949b1e27c
0005ac76-50fe-42f1-8168-8b6fc046e40f
0006811c-ad80-4cd0-a732-04cc983ec8c2
0008f82f-9713-4074-8793-0d266d53930c
000aa34d-c3c0-4176-ad8a-50e801056df1
000b6a38-c69a-4ac9-918b-717a79cbdad2
000b8c7e-4671-4586-9eff-97c0c374126b

In [29]:
from sklearn.utils import shuffle

In [30]:
binary_multilabel = shuffle(binary_multilabel,random_state=0)

In [31]:
for i in range(0,10):
    print(binary_multilabel.index[i][0])

df76ffdf-70d6-4a38-9d60-a1765c18914e
dca1f897-c8bd-4e35-a839-5953ee94d54e
3bec5cd0-76bd-48b1-924a-567bd3361ec0
5eb7cd3c-7631-11e4-a3cb-005056011aef
a67385c3-8562-4dc1-96ba-d96ff215943b
5e35118a-7631-11e4-a3cb-005056011aef
5feb658b-7631-11e4-a3cb-005056011aef
144a86f9-6902-444c-87bc-b389a6f3b275
5e139390-7631-11e4-a3cb-005056011aef
e5741923-bc21-46bd-8832-886706f59e81


df76ffdf-70d6-4a38-9d60-a1765c18914e
dca1f897-c8bd-4e35-a839-5953ee94d54e
3bec5cd0-76bd-48b1-924a-567bd3361ec0
5eb7cd3c-7631-11e4-a3cb-005056011aef
a67385c3-8562-4dc1-96ba-d96ff215943b
5e35118a-7631-11e4-a3cb-005056011aef
5feb658b-7631-11e4-a3cb-005056011aef
144a86f9-6902-444c-87bc-b389a6f3b275
5e139390-7631-11e4-a3cb-005056011aef
e5741923-bc21-46bd-8832-886706f59e81

### Upsample minority classes to address imbalance leading to ~2, 465, 570 rows of data!

Access taxon columns with indexing. 

In [32]:
print("[ENCODING] Taxon min indx:",binary_multilabel.columns[0],"Taxon max indx:",
      binary_multilabel.columns[len(binary_multilabel.columns)-1])

[ENCODING] Taxon min indx: 1 Taxon max indx: 210


In [33]:
binary_multilabel[1].shape

(91771,)

In [34]:
type(binary_multilabel.columns[0])

numpy.int64

In [35]:
### Array with indices to upsample

In [36]:
index=[]
for i in range(0,nb_training_samples):
    index.append(binary_multilabel.index[i][0])
print(len(index))

73416


In [37]:
binary_multilabel[binary_multilabel[1]==1].loc[index].head()

,,,level2taxon_code,1,2,3,4,5,6,7,8,9,10,...,201,202,203,204,205,206,207,208,209,210
content_id,combined_text,title,description,,,,,,,,,,,,,,,,,,,,,
56929879-ef37-4d62-ae36-8985fd738369,administrative justice and tribunals: final progress report this is the final performance report against the commitments in the administrative justice and tribunals strategic work programme. the administrative justice and tribunals strategic work programme 2013 to 2016 sets out the government’s overarching objectives for reforming and improving the administrative justice and tribunal system to ensure it meets the principles of fairness efficiency and accessibility for users. the first progress report was published in june 2014 and we have continued to work across all these areas. this is our final update on how we have delivered the objectives specified in the strategic work programme. related links administrative justice and tribunals: a strategic work programme 2013 16 administrative justice and tribunals annual performance report: 2013 to 2014 administrative justice and tribunals: final progress report ref: isbn 9781474141321 cm 9319 pdf 442kb 45 pages this file may not be suitable for users of assistive technology. request an accessible format. if you use assistive technology (such as a screen reader) and need a version of this document in a more accessible format please email web.comments@justice.gsi.gov.uk . please tell us what format you need. it will help us if you say what assistive technology you use. administrative justice and tribunals: final progress report (print version) ref: isbn 9781474141314 cm 9319 pdf 1010kb 48 pages this file may not be suitable for users of assistive technology. request an accessible format. if you use assistive technology (such as a screen reader) and need a version of this document in a more accessible format please email web.comments@justice.gsi.gov.uk . please tell us what format you need. it will help us if you say what assistive technology you use.,administrative justice and tribunals: final progress report,this is the final performance report against the commitments in the administrative justice and tribunals strategic work programme.,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5fa8e157-7631-11e4-a3cb-005056011aef,proposals to reform judicial review the government response to the joint committee on human rights’ (jchr) 13th report of the 2013 to 2014 session. the government has set out its view on the committee’s recommendations in respect of its proposed reforms to judicial review many of which are being taken forward through the criminal justice and courts bill. the government’s view is that the reforms being taken forward are a proportionate response to the concerns raised in the consultation judicial review – proposals for further reform. the government is clear that judicial review is and must remain a crucial check on the power of the state and should continue to be readily available where it’s necessary in the interests of justice. the reforms the government is pursuing are aimed at speeding up the process for people who have arguable grounds and a genuine case to put. further information government response to the consultation judicial review: proposals for further reform government’s proposals to reform judicial review pdf 253kb 28 pages,proposals to reform judicial review,the government response to the joint committee on human rights’ (jchr) 13th report of the 2013 to 2014 session.,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8ea3bdc9-05ff-4416-8874-248a13bc7610,merger of north sussex and west sussex local justice areas seeks views on merger of 2 local justice areas (ljas) of sussex northern and sussex western into 1 lja to be known as the west sussex lja. the aim of this consultation is to find views on proposals to merge the local justice areas of sussex (northern) and sussex (western) into one new west sussex local justice area. this will give greater flexibility in managing the caseload

In [38]:
# Why are we deleting this?
del binary_multilabel.columns.name

In [ ]:
#TAKES FOREVER TO RUN!
from sklearn.utils import resample

In [ ]:
upsampled_training = pd.DataFrame()
upper = len(binary_multilabel.columns)+1

for taxon in range(1, upper):
    num_samples = binary_multilabel[binary_multilabel[taxon]==1].shape[0] 
    if num_samples<500:
        print("Taxon code:",taxon,"Taxon name:",labels_index[taxon])
        print("SMALL SUPPORT:",num_samples)
        df_minority = binary_multilabel[binary_multilabel[taxon]==1].loc[index]
        if not df_minority.empty:
        # Upsample minority class
            print(df_minority.shape)
            df_minority_upsampled = resample(df_minority, 
                                                 replace=True,     # sample with replacement
                                                 n_samples=(500),    # to match majority class, switch to max_content_freq if works
                                                 random_state=123) # reproducible results
            
            print("FIRST 5 IDs:",[df_minority_upsampled.index[i][0] for i in range(0,5)])

            # Combine majority class with upsampled minority class
            upsampled_training = pd.concat([upsampled_training, df_minority_upsampled])

            # Display new shape
            print("UPSAMPLING:",upsampled_training.shape)

upsampled_training = shuffle(upsampled_training,random_state=0)

Taxon code: 1 Taxon name: Administrative justice reform
SMALL SUPPORT: 11
(9, 210)
FIRST 5 IDs: ['8ea3bdc9-05ff-4416-8874-248a13bc7610', '8ea3bdc9-05ff-4416-8874-248a13bc7610', 'fd0b66df-bab6-4e8a-bd7b-bb12a8ca63ca', '5fa8e157-7631-11e4-a3cb-005056011aef', '5f617c08-7631-11e4-a3cb-005056011aef']
UPSAMPLING: (500, 210)
Taxon code: 2 Taxon name: Adoption, fostering and surrogacy
SMALL SUPPORT: 69


### Doublecheck dataframe contents before merging.

In [ ]:
binary_multilabel.shape

In [ ]:
binary_multilabel.index[91770][0] # final sample before merging.

In [ ]:
binary_multilabel = pd.concat([binary_multilabel, upsampled_training])

In [ ]:
binary_multilabel.index[total_size][0] # first sample of duplicated training data

Do not remove index because the text data lives there.
**TODO** Consider reworking how datasets are set up at some point

In [ ]:
binary_multilabel.to_csv(os.path.join(DATADIR, 'balanced_level2_training_set_sampled.csv.gz'), compression='gzip')

### LOAD OVERSAMPLED DATASET

In [ ]:
balanced_df = pd.read_csv(os.path.join(DATADIR, 'balanced_level2_training_set_sampled.csv.gz'), dtype=object, compression='gzip')

In [ ]:
balanced_df.shape

In [ ]:
#will convert columns to an array of shape
print('Shape of Y multilabel array before train/val/test split:{}'.format(balanced_df[list(balanced_df.columns)].values.shape))

In [ ]:
#dont' overwirte blanced_df as it take sages to read in
balanced_df_taxons = balanced_df.iloc[:,4:215]

In [ ]:
balanced_df_taxons.columns = balanced_df_taxons.columns.astype(int)

In [ ]:
balanced_df_taxons = balanced_df_taxons.astype(int)

In [ ]:
#convert columns to an array. Each row represents a content item, each column an individual taxon
binary_multilabel = balanced_df_taxons[list(balanced_df_taxons.columns)].values
print('Example row of multilabel array {}'.format(binary_multilabel[2]))

In [ ]:
balanced_df.head()

# Format metadata/X

In [ ]:
#extract content_id index to df
meta1 = pd.DataFrame(balanced_df['content_id'])

In [ ]:
print(meta1.shape)
meta1.head()

In [ ]:
metas = ['document_type','first_published_at','publishing_app','primary_publishing_organisation']

In [ ]:
def build_index(x):
    index_dict = {}
    index_dict['index'] = 0
    for i,elem in enumerate(x):
        index_dict[elem] = i+1
    return index_dict

In [ ]:
import time

In [ ]:
#IF THIS FUNCTION TURNS OUT FASTER KEEP
#apply meta data to content
print("STARTED:",time.strftime("%H:%M:%S"))
for meta in metas:
    print("WORKON:",meta)
    meta1[meta] = meta1['content_id'].map(dict(zip(labelled_level2['content_id'], labelled_level2[meta])))
print("FINISHED:",time.strftime("%H:%M:%S"))

In [ ]:
meta1 = meta1.replace(np.nan, '', regex=True) #conver nans to empty strings for labelencoder types
meta1.head()

In [ ]:
def to_cat_to_hot(column):
    doctype_encoder = LabelEncoder()
    new_col = column+"_cat"
    meta1[new_col] = doctype_encoder.fit_transform(meta1[column])
    return to_categorical(meta1[new_col])

dict_of_encodings = {}
for meta in metas:
    if meta != "first_published_at":
        print(meta)
        dict_of_encodings[meta] = to_cat_to_hot(meta)   

In [ ]:
meta1.head()

In [ ]:
meta1['first_published_at'] = pd.to_datetime(meta1['first_published_at'])
print(meta1['first_published_at'].shape)

In [ ]:
first_published = np.array(meta1['first_published_at']).reshape(meta1['first_published_at'].shape[0], 1)

In [ ]:
print(first_published.dtype,first_published.shape,type(first_published))

In [ ]:
dict_of_encodings.keys()

In [ ]:
meta = np.concatenate((dict_of_encodings['document_type'], 
                               dict_of_encodings['primary_publishing_organisation'], 
                               dict_of_encodings['publishing_app']), 
                              axis=1)

In [ ]:
nb_metavars = meta.shape[1]
print(nb_metavars)
print(meta.shape)

### Tokenize text fields

Tokenizer = Class for vectorizing texts, or/and turning texts into sequences (=list of word indexes, where the word of rank i in the dataset (starting at 1) has index i)

In [ ]:
def tokenize(local_tokenizer,input_data,option):
# apply tokenizer to our text data
    data = []
    local_tokenizer.fit_on_texts(input_data)
# list of word indexes, where the word of rank i in the dataset (starting at 1) has index i
    sequences = local_tokenizer.texts_to_sequences(input_data)
    word_index = local_tokenizer.word_index  
    print('Found %s unique tokens.' % len(word_index))
    if option:
        data = pad_sequences(sequences, maxlen= MAX_SEQUENCE_LENGTH)
    else:
        data = tokenizer.sequences_to_matrix(sequences)
    return data

In [ ]:
# True for sequences to matrix, False otherwise.
texts = balanced_df['combined_text']
tokenizer = Tokenizer(num_words=NUM_WORDS)
data = tokenize(tokenizer,texts,False)

titles = balanced_df['title']
tokenizer_tit = Tokenizer(num_words=10000)
onehot_tit = tokenize(tokenizer_tit,titles,True)

descs = balanced_df['description']
tokenizer_desc = Tokenizer(num_words=10000)
onehot_desc = tokenize(tokenizer_desc,descs,True)

In [ ]:
h5f = h5py.File("tokenized_combined_text.hdf5", "w")
h5f.create_dataset('tokenized_combined_text', data=data)
h5f.close()
# data = pd.read_pickle(os.path.join(DATADIR, 'tokenized_combined_text'))

In [ ]:
print('Shape of label tensor:', binary_multilabel.shape)
print('Shape of data tensor:', data.shape)

### Data split
- Training data = 80%
- Development data = 10%
- Test data = 10%

In [ ]:
def split(data,splits):
    l = []
    for (start,end) in splits:
        l.append(data[start:end])
    return [x for x in l]

In [ ]:
print(total_size)

In [ ]:
splits = [(0,-nb_dev_samples),(-nb_dev_samples,-nb_test_samples),(-nb_test_samples,total_size)]
re_split = [(total_size,len(data))]

In [ ]:
x_train, x_dev, x_test = split(data,splits)
x_resampled = split(data,re_split)

In [ ]:
print(x_train.shape,x_resampled.shape)

In [ ]:
x_train = np.concatenate([x_train,x_resampled],axis=0)    

In [ ]:
x_train.shape

In [ ]:
                                                          
meta_train, meta_dev, meta_test = split(metadata,splits)
meta_resampled = split(metadata,re_split)   
meta_train += meta_resampled 
                                                                  
title_train, title_dev, title_test = split(title_data,splits)
title_resampled = split(title_data,re_split)    
title_train += title_resampled
                                                                  
desc_train, desc_dev, desc_test = split(desc_data,splits)
desc_resampled = split(desc_data,re_split)  
desc_train += desc_resampled
                                                                  
y_train, y_dev, y_test = split(labels,splits)
y_resampled = split(labels,re_split)
y_train += y_resampled                                                                  

In [ ]:
print('Shape of x_train:', x_train.shape)
print('Shape of metax_train:', meta_train.shape)
print('Shape of titlex_train:', title_train.shape)
print('Shape of descx_train:', desc_train.shape)
print('Shape of y_train:', y_train.shape)

In [ ]:
print('Shape of x_dev:', x_dev.shape)
print('Shape of meta_dev:', meta_dev.shape)
print('Shape of titlex_dev:', title_dev.shape)
print('Shape of descx_dev:', desc_dev.shape)
print('Shape of y_dev:', y_dev.shape)

In [ ]:
print('Shape of x_test:', x_test.shape)
print('Shape of metax_test:', meta_test.shape)
print('Shape of titlex_test:', title_test.shape)
print('Shape of descx_test:', desc_test.shape)
print('Shape of y_test:', y_test.shape)

### preparing the Embedding layer

NB stopwords haven't been removed yet...

In [ ]:
embedding_layer = Embedding(len(word_index) + 1, 
                            EMBEDDING_DIM, 
                            input_length=MAX_SEQUENCE_LENGTH)

An Embedding layer should be fed sequences of integers, i.e. a 2D input of shape (samples, indices). These input sequences should be padded so that they all have the same length in a batch of input data (although an Embedding layer is capable of processing sequence of heterogenous length, if you don't pass an explicit input_length argument to the layer).

All that the Embedding layer does is to map the integer inputs to the vectors found at the corresponding index in the embedding matrix, i.e. the sequence [1, 2] would be converted to [embeddings[1], embeddings[2]]. This means that the output of the Embedding layer will be a 3D tensor of shape (samples, sequence_length, embedding_dim).

### Estimate class weights for unbalanced datasets.
paramter to model.fit = __class_weight__: Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.

Implement class_weight from sklearn:

- Import the module 

`from sklearn.utils import class_weight`
- calculate the class weight, If ‘balanced’, class weights will be given by n_samples / (n_classes * np.bincount(y)):

`class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)`

- change it to a dict in order to work with Keras.

`class_weight_dict = dict(enumerate(class_weight))`

- Add to model fitting

`model.fit(X_train, y_train, class_weight=class_weight)`

In [ ]:
# class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
# class_weight_dict = dict(enumerate(class_weight))

### Custom loss function

In [ ]:
class WeightedBinaryCrossEntropy(object):

    def __init__(self, pos_ratio):
        neg_ratio = 1. - pos_ratio
        #self.pos_ratio = tf.constant(pos_ratio, tf.float32)
        self.pos_ratio = pos_ratio
        #self.weights = tf.constant(neg_ratio / pos_ratio, tf.float32)
        self.weights = neg_ratio / pos_ratio
        self.__name__ = "weighted_binary_crossentropy({0})".format(pos_ratio)

    def __call__(self, y_true, y_pred):
        return self.weighted_binary_crossentropy(y_true, y_pred)

    def weighted_binary_crossentropy(self, y_true, y_pred):
            # Transform to logits
            epsilon = tf.convert_to_tensor(K.common._EPSILON, y_pred.dtype.base_dtype)
            y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
            y_pred = tf.log(y_pred / (1 - y_pred))

            cost = tf.nn.weighted_cross_entropy_with_logits(y_true, y_pred, self.weights)
            return K.mean(cost * self.pos_ratio, axis=-1)
    
y_true_arr = np.array([0,1,0,1], dtype="float32")
y_pred_arr = np.array([0,0,1,1], dtype="float32")
y_true = tf.constant(y_true_arr)
y_pred = tf.constant(y_pred_arr)

with tf.Session().as_default(): 
    print(WeightedBinaryCrossEntropy(0.5)(y_true, y_pred).eval())
    print(binary_crossentropy(y_true, y_pred).eval())


### difficulty getting global precision/recall metrics . CAUTION interpreting monitoring metrics
fcholltet: "Basically these are all global metrics that were approximated
batch-wise, which is more misleading than helpful. This was mentioned in
the docs but it's much cleaner to remove them altogether. It was a mistake
to merge them in the first place."

In [ ]:
def f1(y_true, y_pred):
    """Use Recall  and precision metrics to calculate harmonic mean (F1 score).

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1 = 2*((precision*recall)/(precision+recall))
    
    return f1

## Training a 1D convnet

### 1. Create model

In [ ]:
NB_CLASSES = y_train.shape[1]
NB_METAVARS = metax_train.shape[1]



sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='wordindex') #MAX_SEQUENCE_LENGTH
embedded_sequences = embedding_layer(sequence_input)
x = Dropout(0.2, name = 'dropout_embedded')(embedded_sequences)

x = Conv1D(128, 5, activation='relu', name = 'conv0')(x)

x = MaxPooling1D(5, name = 'max_pool0')(x)

x = Dropout(0.5, name = 'dropout0')(x)

x = Conv1D(128, 5, activation='relu', name = 'conv1')(x)

x = MaxPooling1D(5 , name = 'max_pool1')(x)

x = Conv1D(128, 5, activation='relu', name = 'conv2')(x)

x = MaxPooling1D(35, name = 'global_max_pool')(x)  # global max pooling

x = Flatten()(x) #reduce dimensions from 3 to 2; convert to vector + FULLYCONNECTED

meta_input = Input(shape=(NB_METAVARS,), name='meta')
meta_hidden = Dense(128, activation='relu', name = 'hidden_meta')(meta_input)
meta_hidden = Dropout(0.2, name = 'dropout_meta')(meta_hidden)


title_input = Input(shape=(titlex_train.shape[1],), name='titles')
title_hidden = Dense(128, activation='relu', name = 'hidden_title')(title_input)
title_hidden = Dropout(0.2, name = 'dropout_title')(title_hidden)

desc_input = Input(shape=(descx_train.shape[1],), name='descs')
desc_hidden = Dense(128, activation='relu', name = 'hidden_desc')(desc_input)
desc_hidden = Dropout(0.2, name = 'dropout_desc')(desc_hidden)

concatenated = concatenate([meta_hidden, title_hidden, desc_hidden, x])

x = Dense(400, activation='relu', name = 'fully_connected0')(concatenated)

x = Dropout(0.2, name = 'dropout1')(x)

x = Dense(NB_CLASSES, activation='sigmoid', name = 'fully_connected1')(x)

# # The Model class turns an input tensor and output tensor into a model
# This creates Keras model instance, will use this instance to train/test the model.
model = Model(inputs=[meta_input, title_input, desc_input, sequence_input], outputs=x)

### 2. Compile model

In [ ]:
# model.compile(loss=WeightedBinaryCrossEntropy(POS_RATIO),
#               optimizer='rmsprop',
#               metrics=['binary_accuracy', f1])

Metric values are recorded at the end of each epoch on the training dataset. If a validation dataset is also provided, then the metric recorded is also calculated for the validation dataset.

All metrics are reported in verbose output and in the history object returned from calling the fit() function. In both cases, the name of the metric function is used as the key for the metric values. In the case of metrics for the validation dataset, the “val_” prefix is added to the key.

You have now built a function to describe your model. To train and test this model, there are four steps in Keras:
1. Create the model by calling the function above
2. Compile the model by calling `model.compile(optimizer = "...", loss = "...", metrics = ["accuracy"])`
3. Train the model on train data by calling `model.fit(x = ..., y = ..., epochs = ..., batch_size = ...)`
4. Test the model on test data by calling `model.evaluate(x = ..., y = ...)`

If you want to know more about `model.compile()`, `model.fit()`, `model.evaluate()` and their arguments, refer to the official [Keras documentation](https://keras.io/models/model/).


In [ ]:
model.summary()

### Tensorboard callbacks /metrics /monitor training

<span style="color:red"> **Size of these files is killing storage during training. Is it histograms?**</span>

In [ ]:
tb = TensorBoard(log_dir='./learn_embedding_logs', histogram_freq=1, write_graph=True, write_images=False)

In [ ]:
CHECKPOINT_PATH = os.path.join(DATADIR, 'model_checkpoint.hdf5')

cp = ModelCheckpoint(
                     filepath = CHECKPOINT_PATH, 
                     monitor='val_loss', 
                     verbose=0, 
                     save_best_only=False, 
                     save_weights_only=False, 
                     mode='auto', 
                     period=1
                    )

In [ ]:
# class Metrics(Callback):
#     def on_train_begin(self, logs={}):
#         self.val_f1s = []
#         self.val_recalls = []
#         self.val_precisions = []
 
#     def on_epoch_end(self, epoch, logs={}):
#         val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
#         val_targ = self.model.validation_data[1]
        
#         self.val_f1s.append(f1_score(val_targ, val_predict, average='micro'))
#         self.val_recalls.append(recall_score(val_targ, val_predict))
#         self.val_precisions.append(precision_score(val_targ, val_predict))
#         print("- val_f1: %f — val_precision: %f — val_recall %f" 
#                 %(f1_score(val_targ, val_predict, average='micro'), 
#                   precision_score(val_targ, val_predict),
#                    recall_score(val_targ, val_predict)))
#         return
 
# metrics = Metrics()

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
#model.fit(x, y, validation_split=0.2, callbacks=[early_stopping])

### 3. Train model

In [ ]:
# metrics callback causes: CCCCCCR55555555511155
# So disable for now
from keras.utils import multi_gpu_model

# Replicates `model` on 8 GPUs.
# This assumes that your machine has 8 available GPUs.
parallel_model = multi_gpu_model(model, gpus=8)
parallel_model.compile(loss=WeightedBinaryCrossEntropy(POS_RATIO),
              optimizer='rmsprop',
              metrics=['binary_accuracy', f1])

# This `fit` call will be distributed on 8 GPUs.
# Since the batch size is 256, each GPU will process 32 samples.
history = parallel_model.fit(
    {'meta': metax_train, 'titles': titlex_train, 'descs': descx_train, 'wordindex': x_train},
    y_train, 
    validation_data=([metax_dev, titlex_dev, descx_dev, x_dev], y_dev), 
    epochs=10, batch_size=128, callbacks=[early_stopping]
)


# history = model.fit(
#     {'meta': metax_train, 'titles': titlex_train, 'descs': descx_train, 'wordindex': x_train},
#     y_train, 
#     validation_data=([metax_dev, titlex_dev, descx_dev, x_dev], y_dev), 
#     epochs=10, batch_size=128, callbacks=[early_stopping]
# )

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, 10)

plt.plot(epochs, loss_values, 'bo', label='Training loss')           
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')      
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()    

f1_values = history_dict['f1']
val_f1_values = history_dict['val_f1']

plt.plot(epochs, f1_values, 'bo', label='Training f1')
plt.plot(epochs, val_f1_values, 'b', label='Validation f1')
plt.title('Training and validation batch-level f1-micro')
plt.xlabel('Epochs')
plt.ylabel('F1-micro')
plt.legend()

plt.show()

### Evaluate model

#### Training metrics

In [ ]:
y_prob = parallel_model.predict([metax_train, titlex_train, descx_train, x_train])

In [ ]:
y_prob.shape

In [ ]:
y_pred = y_prob.copy()
y_pred[y_pred>=P_THRESHOLD] = 1
y_pred[y_pred<P_THRESHOLD] = 0

In [ ]:
f1_score(y_train, y_pred, average='micro')

In [ ]:
#average= None, the scores for each class are returned.
#precision_recall_fscore_support(y_train, y_pred, average=None, sample_weight=None)

In [ ]:
#a = precision_recall_fscore_support(y_train, y_pred, average=None, sample_weight=None)
# pd.DataFrame(list(a))
# f1_byclass = pd.DataFrame((a)[2], columns=['f1'])

# support_byclass = pd.DataFrame((a)[3], columns=['support'])

# f1_byclass = pd.merge(
#     left=f1_byclass, 
#     right=support_byclass, 
#     left_index=True,
#     right_index=True,
#     how='outer', 
#     validate='one_to_one'
# )

# f1_byclass['index_col'] = f1_byclass.index

# f1_byclass['level2taxon'] = f1_byclass['index_col'].map(labels_index).copy()

# print("At p_threshold of {}, there were {} out of {} ({})% taxons with auto-tagged content in the training data"
#       .format(P_THRESHOLD, 
#               f1_byclass.loc[f1_byclass['f1'] > 0].shape[0], 
#               y_pred.shape[1], 
#               (f1_byclass.loc[f1_byclass['f1'] > 0].shape[0]/y_pred.shape[1])*100 ))

In [ ]:
# no_auto_content = f1_byclass.loc[f1_byclass['f1'] == 0]
# no_auto_content = no_auto_content.set_index('level2taxon')

In [ ]:
# no_auto_content['support'].sort_values().plot( kind = 'barh', figsize=(20, 20))

In [ ]:
# classes_predictedto = f1_byclass.loc[f1_byclass['f1'] > 0]
# classes_predictedto = classes_predictedto.set_index('level2taxon') 

In [ ]:
# classes_predictedto.plot.scatter(x='support', y='f1', figsize=(20, 10), xticks=np.arange(0, 9700, 100))

In [ ]:
# classes_predictedto['f1'].sort_values().plot( kind = 'barh', figsize=(20, 20))

In [ ]:
#Calculate globally by counting the total true positives, false negatives and false positives.
precision_recall_fscore_support(y_train, y_pred, average='micro', sample_weight=None) 

In [ ]:
#Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account
precision_recall_fscore_support(y_train, y_pred, average='macro', sample_weight=None)

In [ ]:
#Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account
precision_recall_fscore_support(y_train, y_pred, average='weighted', sample_weight=None)

#### Development set metrics

In [ ]:
y_pred_dev = parallel_model.predict([metax_dev, titlex_dev, descx_dev, x_dev])

In [ ]:
y_pred_dev[y_pred_dev>=P_THRESHOLD] = 1
y_pred_dev[y_pred_dev<P_THRESHOLD] = 0

In [ ]:
#average= None, the scores for each class are returned.
precision_recall_fscore_support(y_dev, y_pred_dev, average=None, sample_weight=None)

In [ ]:
#Calculate globally by counting the total true positives, false negatives and false positives.
precision_recall_fscore_support(y_dev, y_pred_dev, average='micro', sample_weight=None) 

In [ ]:
#Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account
precision_recall_fscore_support(y_dev, y_pred_dev, average='macro', sample_weight=None)

In [ ]:
#Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account
precision_recall_fscore_support(y_dev, y_pred_dev, average='weighted', sample_weight=None)